<a href="https://colab.research.google.com/github/roitraining/SparkforDataScientists/blob/Development/Ch08_DeepLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning With Apache Spark

### Load Spark-DL into the notebook.


In [0]:
# pip install tensorflow==1.15
import os
SUBMIT_ARGS = "--packages databricks:spark-deep-learning:1.0.0-spark2.3-s_2.11 pyspark-shell"
os.environ["PYSPARK_SUBMIT_ARGS"] = SUBMIT_ARGS

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("DL with Spark Deep Cognition").getOrCreate()
sc = spark.sparkContext
print(sc)

### Load an image.

Let's do a sample image recognition using Deep Learning, but loading some images into Spark.

In [0]:
# !curl -O http://download.tensorflow.org/example_images/flower_photos.tgz
# !tar xzf flower_photos.tgz
# ! rm flower_photos.tgz
# !mkdir flower_photos/sample
# !cp flower_photos/daisy/100080576_f52e8ee070_n.jpg flower_photos/sample/
# !cp flower_photos/daisy/10140303196_b88d3d6cec.jpg flower_photos/sample/
# !cp flower_photos/tulips/100930342_92e8746431_n.jpg flower_photos/sample/

### Display a few images.

In [0]:
from IPython.display import display_png, Image, display
import os

path = '/home/student/ROI/Spark/flower_photos/'
files = os.listdir(path + 'sample/')
images = [Image(filename = path + 'sample/' + file, format = 'png') for file in files if file.endswith('.jpg')]
for ea in images:
    display_png(ea)

print ('Done')

### Read images into a Spark DataFrame.

In [0]:
from pyspark.ml.image import ImageSchema
image_df = ImageSchema.readImages(path + "sample/")
image_df.printSchema()
#image_df.take(1)


### We need to install TensorFlow and Keras, but for this demo we need an older version of TensorFlow.
We split up the dataets as we normally would into training and testing sets.

In [0]:
# !pip install tensorflow==1.15
# !pip install keras

from pyspark.ml.image import ImageSchema
from pyspark.sql.functions import lit
from sparkdl.image import imageIO

tulips_df = ImageSchema.readImages(path + "tulips/").withColumn("label", lit(1))
daisy_df = imageIO.readImagesWithCustomFn(path + "daisy/", decode_f=imageIO.PIL_decode).withColumn("label", lit(0))

tulips_train, tulips_test, _ = tulips_df.randomSplit([0.1, 0.05, 0.85])  # use larger training sets (e.g. [0.6, 0.4] for getting more images)
daisy_train, daisy_test, _ = daisy_df.randomSplit([0.1, 0.05, 0.85])     # use larger training sets (e.g. [0.6, 0.4] for getting more images)

train_df = tulips_train.unionAll(daisy_train)
test_df = tulips_test.unionAll(daisy_test)

display(train_df)
#print(train_df.take(1))

# Under the hood, each of the partitions is fully loaded in memory, which may be expensive.
# This ensure that each of the paritions has a small size.
#train_df = train_df.repartition(100)
#test_df = test_df.repartition(100)
print ('Done')

### Let’s train the model.

In [0]:
# nano /etc/java-11-openjdk/accessibility.properties
# comment out assistive_technologies=org.GNOME.Accessibility.AtkWrapper
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from sparkdl import DeepImageFeaturizer 

featurizer = DeepImageFeaturizer(inputCol="image", outputCol="features", modelName="InceptionV3")
lr = LogisticRegression(maxIter=10, regParam=0.05, elasticNetParam=0.3, labelCol="label")
p = Pipeline(stages=[featurizer, lr])

p_model = p.fit(train_df)
print ('Done')

In [0]:
#p_model.save('/home/student/ROI/flower.model')

In [0]:
# from pyspark.ml import PipelineModel
# p_model2 = PipelineModel.load('/home/student/ROI/flower.model')

### Look at how well the model performs.

In [0]:
predictions = p_model.transform(test_df)
predictions.cache()

In [0]:
#from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql.types import IntegerType, DoubleType
# evaluator = MultiClassificationEvaluator()
# evaluator.evaluate(predictions
x = predictions.select("prediction", "label").withColumn("label", predictions["label"].cast(DoubleType()))
#print(x.take(10))
metrics = MulticlassMetrics(x.rdd)
print(metrics.confusionMatrix())


### We can take look at where we are making mistakes:

In [0]:
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import expr
from pyspark.sql.functions import *
from pyspark.sql.types import *

def _p1(v):
    return float(v.array[1])
take_one = udf(_p1, DoubleType())

df = predictions.withColumn("p", take_one(predictions.probability))
wrong_df = df.orderBy(expr("abs(p - label)"), ascending=False)
#wrong_df.select("image.origin", "p", "label").show(10)
wrong_df.createOrReplaceTempView('wrong')

print('Done')

In [0]:
x = spark.sql('select image.origin as origin, p, label from wrong order by p desc').collect()
print(x)

In [0]:
#print(x)
for label in range(2):
    for file in [f for f in x if f.label == label][:3]:
        print (file.origin[5:])
        display_png(Image(filename = file.origin[5:], format = 'png') )



### Working with general tensors

Deep Learning Pipelines also provides ways to apply models with tensor inputs
(up to 2 dimensions), written in popular deep learning libraries:

* TensorFlow graphs
* Keras models

In this article, we will focus only on the Keras models. The `KerasTransformer`
applies a TensorFlow-backed Keras model to tensor inputs of up to 2 dimensions.
It loads a Keras model from a given model file path and applies the model to a
column of arrays (where an array corresponds to a Tensor), outputting a column
of arrays.

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from sparkdl import KerasTransformer
from keras.models import Sequential
from keras.layers import Dense
import numpy as np

# Generate random input data
num_features = 10
num_examples = 100
input_data = [{"features" : np.random.randn(num_features).astype(float).tolist()} for i in range(num_examples)]
schema = StructType([ StructField("features", ArrayType(FloatType()), True)])
input_df = spark.createDataFrame(input_data, schema)

# Create and save a single-hidden-layer Keras model for binary classification
# NOTE: In a typical workflow, we'd train the model before exporting it to disk,
# but we skip that step here for brevity
model = Sequential()
model.add(Dense(units=20, input_shape=[num_features], activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))
model_path = "simple-binary-classification"
model.save(model_path)

# Create transformer and apply it to our input data
transformer = KerasTransformer(inputCol="features", outputCol="predictions", modelFile=model_path)
final_df = transformer.transform(input_df)
print('Done')

In [0]:
final_df.show()

### Deploying Models in SQL

One way to productionize a model is to deploy it as a Spark SQL User Defined Function, which allows anyone who knows SQL to use it. Deep Learning Pipelines provides mechanisms to take a deep learning model and register a Spark SQL User Defined Function (UDF). In particular, Deep Learning Pipelines 0.2.0 adds support for creating SQL UDFs from Keras models that work on image data.

The resulting UDF takes a column (formatted as a image struct “SpImage”) and produces the output of the given Keras model; e.g. for Inception V3, it produces a real valued score vector over the ImageNet object categories.

In [0]:
from keras.applications import InceptionV3
from sparkdl.udf.keras_image_model import registerKerasImageUDF

registerKerasImageUDF("inceptionV3_udf", InceptionV3(weights="imagenet"))

In Keras workflows dealing with images, it’s common to have preprocessing steps before the model is applied to the image. If our workflow requires preprocessing, we can optionally provide a preprocessing function to UDF registration. The preprocessor should take in a filepath and return an image array; below is a simple example.

In [0]:
from keras.applications import InceptionV3
from sparkdl.udf.keras_image_model import registerKerasImageUDF

def keras_load_img(fpath):
    from keras.preprocessing.image import load_img, img_to_array
    import numpy as np
    img = load_img(fpath, target_size=(299, 299))
    return img_to_array(img).astype(np.uint8)

registerKerasImageUDF("inceptionV3_udf_with_preprocessing", InceptionV3(weights="imagenet"), keras_load_img)
print('Done')

Once a UDF has been registered, it can be used in a SQL query:

In [0]:
from pyspark.ml.image import ImageSchema

image_df = ImageSchema.readImages("flower_photos/sample/")
image_df.registerTempTable("sample_images")

In [0]:
display(image_df)

### Once a data scientist builds the desired model, Deep Learning Pipelines makes it simple to expose it as a function in SQL, so anyone in their organization can use it.

In [0]:
display(spark.sql("SELECT inceptionV3_udf_with_preprocessing(image) as predictions from sample_images"))
#.show(truncate=False)

## Code below does not work

### Spark prediction pipeline using InceptionV3.

In [0]:
from sparkdl import DeepImagePredictor
# Read images using Spark
image_df = ImageSchema.readImages("flower_photos/sample/")
predictor = DeepImagePredictor(inputCol="image", outputCol="predicted_labels", modelName="InceptionV3", decodePredictions=True, topK=10)
predictions_df = predictor.transform(image_df)
print ('Done')

### Let's take a look to the predictions dataframe:

In [0]:
predictions_df.select("predicted_labels").show(truncate=False,n=3)

### Notice that the 'predicted_labels' column shows 'daisy' as a high probability class for all of sample flowers using this base model, for some reason the tulip was closer to a picket fence than to a flower (maybe because of the background of the photo). 

However, as can be seen from the differences in the probability values, the
neural network has the information to discern the two flower types. Hence our
transfer learning example above was able to properly learn the differences
between daisies and tulips starting from the base model.

Let’s see how well our model discern the type of the flower:

In [0]:
df = p_model.transform(image_df)

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
def _p1(v):
    return float(v.array[1])
take_one = udf(_p1, DoubleType())

In [0]:
# 100930342_92e8746431_n.jpg not a daisy
df.select("image.origin",(1-take_one(df.probability)).alias("p_daisy")).show(truncate=False)

## For Keras 

To use the transformer, we first need to have a Keras model stored as a file. For this notebook we'll just save the Keras built-in InceptionV3 model instead of training one.

In [0]:
from keras.applications import InceptionV3
model = InceptionV3(weights="imagenet")
model.save('model-full.h5')
print('Done')

### Now we will create a Keras transformer but first we will preprocess the images to work with it.

In [0]:
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.image import img_to_array, load_img
import numpy as np
from pyspark.sql.types import StringType
from sparkdl import KerasImageFileTransformer

def loadAndPreprocessKerasInceptionV3(uri):
  # this is a typical way to load and prep images in keras
    image = img_to_array(load_img(uri, target_size=(299, 299)))  # image dimensions for InceptionV3
    image = np.expand_dims(image, axis=0)
    return preprocess_input(image)

transformer = KerasImageFileTransformer(inputCol="uri", outputCol="predictions",
                                        modelFile='model-full.h5',  # local file path for model
                                        imageLoader=loadAndPreprocessKerasInceptionV3,
                                        outputMode="vector")
print('Done')

### We will read now the images and load them into a Spark Dataframe and them use our transformer to apply the model into the images:

In [0]:
fs = !ls flower_photos/sample/*.jpg

In [0]:
fs

In [0]:
uri_df = spark.createDataFrame(fs, StringType()).toDF("uri")

In [0]:
uri_df.show()

In [0]:
#dir(transformer)
keras_pred_df = transformer.transform(uri_df)

In [0]:
keras_pred_df.select("uri", "predictions").show()